In [14]:
!wget http://www.platform.ai/files/dogscats.zip

--2017-04-30 09:35:49--  http://files.fast.ai/data/dogscats.zip
Resolving www.platform.ai (www.platform.ai)... 67.205.12.187
Connecting to www.platform.ai (www.platform.ai)|67.205.12.187|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 861515103 (822M) [application/zip]
Saving to: ‘dogscats.zip’

dogscats.zip        100%[===================>] 821.60M  14.0MB/s    in 65s     

2017-04-30 09:36:56 (12.7 MB/s) - ‘dogscats.zip’ saved [861515103/861515103]



I ran unzip command in terminal since I have experience that long output line will cause jupyter to freeze.

In [ ]:
!unzip dogscats.zip

In [2]:
%mkdir -p dogscats/test/unknown

In [ ]:
!rm dogscats.zip
%cd dogscats/test1
%mv *.jpg /output/dogscats/test/unknown

In [3]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

Using TensorFlow backend.


In [38]:
from sklearn.utils import shuffle

In [64]:
path = '/output/dogscats/'
#sample_path = '/output/dogscats/sample'
batch_size = 32

From https://github.com/ypwhs/dogs_vs_cats

In [33]:
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(path+'train', image_size, shuffle=False, 
                                              batch_size=batch_size)
    valid_generator = gen.flow_from_directory(path+'valid', image_size, shuffle=False, 
                                              batch_size=batch_size)
    test_generator = gen.flow_from_directory(path+'test', image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)

    train = model.predict_generator(train_generator, train_generator.nb_sample)
    valid = model.predict_generator(valid_generator, valid_generator.nb_sample)
    test = model.predict_generator(test_generator, test_generator.nb_sample)
    with h5py.File("gap_%s.h5"%MODEL.__name__) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("valid", data=valid)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
        h.create_dataset("val_label", data=valid_generator.classes)

**Pre-compte feature data**

In [34]:
%time write_gap(ResNet50, (224, 224))

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
CPU times: user 11min 26s, sys: 54.4 s, total: 12min 20s
Wall time: 16min 18s


In [35]:
%time write_gap(Xception, (299, 299), xception.preprocess_input)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
CPU times: user 16min 42s, sys: 1min 26s, total: 18min 9s
Wall time: 20min 32s


In [36]:
%time write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
CPU times: user 25min 25s, sys: 1min 10s, total: 26min 35s
Wall time: 21min 54s


**Combine output features of ResNet50, Xception and InceptionV3**

In [39]:
X_train = []
X_valid = []
X_test = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_valid.append(np.array(h['valid']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])
        y_valid = np.array(h['val_label'])

X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [40]:
X_train.shape

(23000, 6144)

In [63]:
y_train.shape

(23000,)

#Fine-tunning Model 1#

In [41]:
input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

from keras.optimizers import Nadam
model.compile(optimizer=Nadam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [42]:
model.fit(X_train, y_train, batch_size=batch_size*4, nb_epoch=1, validation_data=(X_valid,y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 5s - loss: 0.0381 - acc: 0.9857 - val_loss: 0.0177 - val_acc: 0.9920

In [44]:
model.fit(X_train, y_train, batch_size=batch_size*4, nb_epoch=1, validation_data=(X_valid,y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 1s - loss: 0.0188 - acc: 0.9938 - val_loss: 0.0189 - val_acc: 0.9935

In [46]:
model.optimizer.lr = 1e-5
model.fit(X_train, y_train, batch_size=batch_size*4, nb_epoch=1, validation_data=(X_valid,y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 1s - loss: 0.0146 - acc: 0.9950 - val_loss: 0.0183 - val_acc: 0.9950

In [61]:
model.optimizer.lr = 1e-6
model.fit(X_train, y_train, batch_size=batch_size*4, nb_epoch=1, validation_data=(X_valid,y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 1s - loss: 0.0143 - acc: 0.9953 - val_loss: 0.0178 - val_acc: 0.9950

In [450]:
model.optimizer.lr = 1e-6
model.fit(X_train, y_train, batch_size=batch_size*4, nb_epoch=1, validation_data=(X_valid,y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 1s - loss: 0.0108 - acc: 0.9961 - val_loss: 0.0194 - val_acc: 0.9925

In [445]:
model.save_weights('model_5.h5')

In [456]:
model.load_weights('model_5.h5')

**Pseudo-labeling**

In [481]:
y_pseudo = model.predict(X_test, batch_size=batch_size)

In [489]:
i_trn = 0
i_test = 0

# iterate through 800 mini-batch
num_iter = 600*2
# mini-batch size
size_trn = 48
size_test = 16
num_batch_per_epoch_trn = int(X_train.shape[0]/size_trn)
num_batch_per_epoch_test = int(X_test.shape[0]/size_test)
index_trn = np.random.permutation(num_batch_per_epoch_trn)
index_test = np.random.permutation(num_batch_per_epoch_test)
for i in range(num_iter):
    i_trn = index_trn[i%num_batch_per_epoch_trn]
    i_test = index_test[i%num_batch_per_epoch_test]
    
    comb_features = np.concatenate((X_train[(size_trn*i_trn):size_trn*(i_trn+1)],
                                   X_test[(size_test*i_test):size_test*(i_test+1)]),axis=0)
    comb_labels = np.concatenate((y_train[(size_trn*i_trn):size_trn*(i_trn+1)],
                                 y_pseudo_r[(size_test*i_test):size_test*(i_test+1)]), axis=0)
    
    model.train_on_batch(comb_features, comb_labels)
    
    if (i+1)%num_batch_per_epoch_trn == 0:
        index_trn = np.random.permutation(num_batch_per_epoch_trn)
    if (i+1)%num_batch_per_epoch_test == 0:
        index_test = np.random.permutation(num_batch_per_epoch_test)

In [483]:
model.optimizer.lr = 0
model.fit(X_train, y_train, batch_size=batch_size*4, nb_epoch=1, validation_data=(X_valid,y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 1s - loss: 0.0057 - acc: 0.9980 - val_loss: 0.0167 - val_acc: 0.9940

#Fine-tunning Model 2#

In [488]:
input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(512, activation='sigmoid')(x) # this should be relu
x = Dropout(0.8)(x)
x = Dense(1, activation='sigmoid')(x)
model2 = Model(input_tensor, x)

from keras.optimizers import Nadam
model2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [472]:
model2.fit(X_train, y_train, batch_size=batch_size*2, nb_epoch=3, validation_data=(X_valid,y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/3
23000/23000 [==============================] - 8s - loss: 0.0334 - acc: 0.9887 - val_loss: 0.0177 - val_acc: 0.9945

In [473]:
model2.optimizer.lr = 1e-5
model2.fit(X_train, y_train, batch_size=batch_size*2, nb_epoch=2, validation_data=(X_valid,y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/2
23000/23000 [==============================] - 2s - loss: 0.0204 - acc: 0.9943 - val_loss: 0.0220 - val_acc: 0.9920

In [490]:
model.optimizer.lr = 1e-9
model.fit(X_train, y_train, batch_size=batch_size*2, nb_epoch=1, validation_data=(X_valid,y_valid))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 2s - loss: 0.0091 - acc: 0.9968 - val_loss: 0.0397 - val_acc: 0.9905

In [474]:
model2.save_weights('model3_1.h5')

In [480]:
model2.load_weights('model3_1.h5')

# Submit

In [469]:
y_pred = model2.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

12320/12500 [============================>.] - ETA: 0s   

In [367]:
test_batches = ImageDataGenerator().flow_from_directory(path+'test', shuffle=False, batch_size=16, class_mode=None)
filenames = test_batches.filenames

Found 12500 images belonging to 1 classes.


In [369]:
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

In [470]:
subm = np.stack([ids,y_pred.reshape(-1)], axis=1)

submission_file_name = 'subm_re50_incepV3_Xcep_clip995_Ver2_1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')

**Average submissions**

In [ ]:
from numpy import genfromtxt
import numpy as np
import pandas as pd

In [ ]:
sub1 = genfromtxt('subm_re50_incepV3_Xcep_clip995.csv', delimiter=',')
sub2 = genfromtxt('subm_re50_incepV3_Xcep_clip995_wo_pseudoLabeling.csv', delimiter=',')
sub3 = genfromtxt('subm_re50_incepV3_Xcep_clip995_Ver2.csv', delimiter=',')
sub4 = genfromtxt('subm_re50_incepV3_Xcep_clip995_Ver3.csv', delimiter=',')

In [ ]:
sub1 = sub1[1:]
sub2 = sub2[1:]
sub3 = sub3[1:]
sub4 = sub4[1:]

In [ ]:
sub_avg_pred = (sub1[:,1] + sub2[:,1] + sub3[:,1] + sub4[:,1])/4.
#sub_avg_pred = (0.5*sub1[:,1] + sub2[:,1] + sub3[:,1] + sub4[:,1])/3.5

In [ ]:
sub_avg = sub1
sub_avg[:,1] = sub_avg_pred

In [ ]:
submission_file_name = 'arith_mean_subm.csv'
np.savetxt(path+submission_file_name, sub_avg, fmt='%d,%.5f', header='id,label', comments='')